In [5]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load CSV data
df = pd.read_csv('2010_Chronic_Conditions.csv')
df.head()

,BENE_SEX_IDENT_CD,BENE_AGE_CAT_CD,CC_ALZHDMTA,CC_CANCER,CC_CHF,CC_CHRNKIDN,CC_COPD,CC_DEPRESSN,CC_DIABETES,CC_ISCHMCHT,...,BENE_COUNT_PC_LT_12,AVE_MO_EN_PC_LT_12,BENE_COUNT_PC_EQ_12,BENE_COUNT_PD_LT_12,AVE_MO_EN_PD_LT_12,AVE_PDE_CST_PD_LT_12,AVE_PDE_PD_LT_12,BENE_COUNT_PD_EQ_12,AVE_PDE_CST_PD_EQ_12,AVE_PDE_PD_EQ_12
0,1,1,NaN,NaN,0,0,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,7230.0,49.877
1,1,1,NaN,NaN,0,0,NaN,NaN,0,0,...,NaN,NaN,NaN,42.0,5.952,2165.0,24.81,203.0,7163.0,73.404
2,1,1,NaN,NaN,0,0,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,4007.0,51.481
3,1,1,NaN,NaN,0,0,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209.0,9021.0,75.732
4,1,1,NaN,NaN,0,0,NaN,NaN,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,5925.0,61.591


In [4]:
# Demographic columns
demographic_cols = ['BENE_SEX_IDENT_CD', 'BENE_AGE_CAT_CD']

# Utilization metrics 
utilization_cols = [
    'AVE_IP_ADM_PA_EQ_12',     # Inpatient admits
    'AVE_SNF_DAYS_PA_EQ_12',   # Skilled nursing facility days
    'AVE_CA_VST_PB_EQ_12',     # Physician visits
    'AVE_OP_VST_PB_EQ_12',     # Outpatient visits
    'AVE_PDE_PD_EQ_12'         # Prescription fills
]

In [6]:
udf = df[utilization_cols].fillna(0)

scaler = MinMaxScaler()

df_scaled = pd.DataFrame(scaler.fit_transform(udf), columns=udf.columns)
df_scaled['Total Utilization'] = (df_scaled['AVE_IP_ADM_PA_EQ_12'] +
                          df_scaled['AVE_SNF_DAYS_PA_EQ_12'] +
                          df_scaled['AVE_CA_VST_PB_EQ_12'] +
                          df_scaled['AVE_OP_VST_PB_EQ_12'] +
                          df_scaled['AVE_PDE_PD_EQ_12']
                          )

df_scaled.rename(columns={
    'AVE_IP_ADM_PA_EQ_12': 'Inpatient_Admissions',
    'AVE_SNF_DAYS_PA_EQ_12': 'SNF_Days',
    'AVE_CA_VST_PB_EQ_12': 'Clinic_Visits',
    'AVE_OP_VST_PB_EQ_12': 'Outpatient_Visits',
    'AVE_PDE_PD_EQ_12': 'Prescription_Days'
}, inplace=True)


In [7]:
df_scaled[demographic_cols] = df[demographic_cols]

In [8]:
sex_map = {1: 'Male', 2: 'Female'}
agegrp_map = {
    1: 'Under 65', 2: '65-69', 3: '70-74', 4: '75-79',
    5: '80-84', 6: '85+'
}

df_scaled['BENE_SEX_IDENT_CD'] = df_scaled['BENE_SEX_IDENT_CD'].map(sex_map)
df_scaled['BENE_AGE_CAT_CD'] = df_scaled['BENE_AGE_CAT_CD'].map(agegrp_map)

In [9]:
df_scaled

,Inpatient_Admissions,SNF_Days,Clinic_Visits,Outpatient_Visits,Prescription_Days,Total Utilization,BENE_SEX_IDENT_CD,BENE_AGE_CAT_CD
0,0.159450,0.102909,0.356004,0.260553,0.285543,1.164459,Male,Under 65
1,0.138306,0.142654,0.314862,0.392316,0.420234,1.408372,Male,Under 65
2,0.140768,0.047205,0.445315,0.280507,0.294726,1.208520,Male,Under 65
3,0.189718,0.154639,0.389004,0.381911,0.433562,1.548834,Male,Under 65
4,0.195945,0.000000,0.482277,0.329667,0.352605,1.360495,Male,Under 65
...,...,...,...,...,...,...,...,...
21998,0.434468,0.737148,0.141504,0.342933,0.621592,2.277644,Female,85+
21999,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Female,85+
22000,0.362056,0.000000,0.245395,0.479674,0.793198,1.880324,Female,85+
22001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Female,85+


In [10]:
df_grouped = df_scaled.groupby('BENE_AGE_CAT_CD').mean(numeric_only=True).reset_index()
df_grouped

,BENE_AGE_CAT_CD,Inpatient_Admissions,SNF_Days,Clinic_Visits,Outpatient_Visits,Prescription_Days,Total Utilization
0,65-69,0.137156,0.066661,0.379375,0.308364,0.330635,1.222191
1,70-74,0.139697,0.082022,0.393681,0.297099,0.324639,1.237139
2,75-79,0.142236,0.105242,0.378043,0.282963,0.305662,1.214146
3,80-84,0.143779,0.130342,0.353245,0.271073,0.303431,1.201870
4,85+,0.146602,0.167938,0.293109,0.252443,0.297745,1.157837
5,Under 65,0.172132,0.061778,0.382894,0.369799,0.397135,1.383739


In [11]:
json_preview = df_grouped.to_json(orient='records', indent=2)
print(json_preview)

[
  {
    "BENE_AGE_CAT_CD":"65-69",
    "Inpatient_Admissions":0.1371561922,
    "SNF_Days":0.0666605974,
    "Clinic_Visits":0.3793753808,
    "Outpatient_Visits":0.3083638025,
    "Prescription_Days":0.330634667,
    "Total Utilization":1.22219064
  },
  {
    "BENE_AGE_CAT_CD":"70-74",
    "Inpatient_Admissions":0.1396969572,
    "SNF_Days":0.0820221332,
    "Clinic_Visits":0.3936808899,
    "Outpatient_Visits":0.297099428,
    "Prescription_Days":0.3246390943,
    "Total Utilization":1.2371385026
  },
  {
    "BENE_AGE_CAT_CD":"75-79",
    "Inpatient_Admissions":0.1422360769,
    "SNF_Days":0.105241991,
    "Clinic_Visits":0.3780429087,
    "Outpatient_Visits":0.2829631089,
    "Prescription_Days":0.3056619372,
    "Total Utilization":1.2141460228
  },
  {
    "BENE_AGE_CAT_CD":"80-84",
    "Inpatient_Admissions":0.1437785258,
    "SNF_Days":0.1303418066,
    "Clinic_Visits":0.353245288,
    "Outpatient_Visits":0.2710730147,
    "Prescription_Days":0.3034309286,
    "Total Utiliza